# <center><span style="color:orange">MAIN TRANSACTIONS TABLE</span></center>

For this project I used a private dataset from a DB.</br>
For the presentation purposes I will use a randomly generated dataset.

In [2]:
## Randomly generate a dataset, later we will add quantity for the sake of making it similar to real life cases
import string
import random
import random
import time
import numpy as np
import pandas as pd

def id_generator(size=6, chars=string.ascii_uppercase + string.digits):
    return ''.join(random.choice(chars) for _ in range(size))

def get_rand_date(start, end, time_format, prop):

    stime = time.mktime(time.strptime(start, time_format))
    etime = time.mktime(time.strptime(end, time_format))

    ptime = stime + prop * (etime - stime)

    return time.strftime(time_format, time.localtime(ptime))

def random_date(start, end, prop):
    return get_rand_date(start, end, '%m-%d-%Y', prop)

def get_random_table():
    ind = list(range(5000000))
    qty = []
    txn = []
    date = []
    user = []
    item = []
    user_x = [id_generator(3) for x in range(30000)]
    item_x = [id_generator() for x in range(50000)]
    for i in ind:
        txn.append(np.random.randint(1000, 5000))
        qty.append(np.random.randint(1,500))
        date.append(random_date("01-01-2000", "01-01-2022", random.random()))
        user.append(user_x[np.random.randint(len(user_x))])
        item.append(item_x[np.random.randint(len(item_x))])

    df = pd.DataFrame({"transaction_id": txn, "quantity": qty, "date": date, "user": user, "item": item}, index=ind)
    df["date"] = pd.to_datetime(df["date"])

    return df

random_dataset = get_random_table()
random_dataset.to_csv("random_data.csv")    # save for later to use the same randomly generated dataset
random_dataset.head(10)

,transaction_id,quantity,date,user,item
0,3742,406,2010-06-21,YGC,UUW0JU
1,4232,455,2020-06-13,D3J,TRNNMO
2,3527,260,2005-10-16,70U,VTFM40
3,3756,50,2005-03-04,I94,7L8TEC
4,4908,46,2017-04-26,6NS,WB73LO
5,2767,262,2010-09-20,KCZ,LS9997
6,3929,421,2001-03-25,FDA,YGSLOS
7,2069,128,2019-01-05,76N,6VQVAN
8,2610,226,2019-02-23,PAO,7CNPL0
9,3300,475,2015-06-12,W03,I9EQEW


In [3]:
## or you can use a dataset from DB
#import pymssql as sql

## Database connection
#sql_connection = sql.connect(host='0.0.0.0', user='ABCDE', password='ABCDE', database='ABCDE')

In [4]:
column_name_query = """
                    SELECT
                        name as COLUMN_NAMES
                    FROM
                        sys.columns
                    WHERE
                        object_id = OBJECT_ID('ABCDE.MAIN_TRANSACTION_TABLE')
                    """

<span style="color:orange">Transaction table column names:</span>

In [5]:
## Increase displayed rows to see all column names
pd.options.display.max_rows = 100

#column_names = pd.read_sql(column_name_query, sql_connection)

In [6]:
## Return default display options

pd.reset_option("^display")

<span style="color:orange">We selected related columns which are:</span>

| Column Name    | Goal                        |   | Column Name         | Goal                       |
| -----------    | --------------------------- |---| ------------------- | -------------------------- |
| TRANSACTION ID | Index<br/>aggregation       |   | ITEM                | Main target</br>(recommend)|
| DATE           | Filter<br/>recent           |   | QUANTITY            | Multiply row               |
| USER           | Check<br/>inventory         |   |                     |                            |

<span style="color:orange">First, we retrieve the full table with selected columns.</span>

In [7]:
full_table_query =  """
                    SELECT
                         *
                    FROM
                        ABCDE.MAIN_TRANSACTION_TABLE
                    """

In [8]:
#full_table = pd.read_sql(full_table_query, sql_connection)

## Change date and id column types
#full_table["id"] = full_table["id"].astype(int)
#full_table["date"] = pd.to_datetime(full_table["date"], format="%Y%m%d")

## OUR FULL TABLE WILL BE (for the sake of privacy) RANDOMLY GENERATED DATASET
full_table = random_dataset
full_table

,transaction_id,quantity,date,user,item
0,3742,406,2010-06-21,YGC,UUW0JU
1,4232,455,2020-06-13,D3J,TRNNMO
2,3527,260,2005-10-16,70U,VTFM40
3,3756,50,2005-03-04,I94,7L8TEC
4,4908,46,2017-04-26,6NS,WB73LO
...,...,...,...,...,...
4999995,1920,456,2002-10-30,3X7,6LBZL4
4999996,2708,351,2004-06-24,F02,1VE6ML
4999997,1693,392,2016-02-08,KDV,2RT9FE
4999998,2529,197,2021-09-02,IC2,V9Y5YX


<span style="color:orange">We will use this data frame as CSV.</span>

<span style="color:orange">Now, we will filter our data to include only last 5 years.</span>

In [9]:
from datetime import datetime
from dateutil.relativedelta import relativedelta

five_years_ago = datetime.now() - relativedelta(years=5)

full_five_years_table = full_table[full_table["date"] > five_years_ago].sort_values("date").reset_index(drop=True)
full_five_years_table

,transaction_id,quantity,date,user,item
0,4009,162,2017-06-07,N6D,ZFNBON
1,1040,21,2017-06-07,Q4C,FVTSKM
2,1655,418,2017-06-07,58F,6IG9QR
3,3584,257,2017-06-07,T8D,IM8NYC
4,4654,367,2017-06-07,YOL,RP9YYU
...,...,...,...,...,...
1037843,4550,15,2021-12-31,LL3,0IHPNF
1037844,3226,196,2021-12-31,C51,BHNO0D
1037845,1631,393,2021-12-31,Q8U,SYP35J
1037846,4864,46,2021-12-31,S1H,MM4O5L


<span style="color:orange">"full_table" has 4,000,000 (4M) rows and "full_five_years_table" has 1,037,848 (~1M) rows.</br></br>Next, we will drop rows with "item" that have not been sold for 2 years as passive items.</span>

In [10]:
two_years_ago = datetime.now() - relativedelta(years=2)

full_two_years_table = full_table[full_table["date"] > two_years_ago].sort_values("date").reset_index(drop=True)

active_items = full_two_years_table["item"].drop_duplicates()
active_items = active_items.to_list()
len(active_items)

49965

<span style="color:orange">We have ~49k active items.</br></br>And we will filter the data frame to include only active items.</span>

In [11]:
active_five_years_table = full_five_years_table[full_five_years_table["item"].isin(active_items)].sort_values("date").reset_index(drop=True)
active_five_years_table

,transaction_id,quantity,date,user,item
0,4009,162,2017-06-07,N6D,ZFNBON
1,3305,182,2017-06-07,PLA,BIRCAJ
2,4123,217,2017-06-07,HGX,GSX66F
3,2059,451,2017-06-07,LGK,PV929K
4,4383,261,2017-06-07,3DX,2HH9BB
...,...,...,...,...,...
1037365,2482,426,2021-12-31,O97,DFDSMS
1037366,4920,190,2021-12-31,S3V,CFU0O0
1037367,4096,359,2021-12-31,MMH,WDHG0I
1037368,2377,478,2021-12-31,AF5,UITYV5


<span style="color:orange">"full_five_years_table" has 1,038,367 (~1M) rows and "active_five_years_table" has 1,037,370 (~1M) rows.</br></br>Above data frame is filtered with 2 operations, so we will save this data frame as CSV for later use.</span>

In [12]:
active_five_years_table.to_csv("active_five_years_table.csv", index=False)

## <center><span style="color:orange">Majority Analysis</span></center>
<span style="color:orange">We will try to find which item set among active items can represent 90-95% (almost all) of the data.</span>

In [13]:
## Let's take prepared data frame and aggregate to see which item is sold most
majority_items = active_five_years_table.groupby("item").agg({"quantity":"sum"})
majority_items = majority_items.sort_values("quantity", ascending=False).reset_index()

## Then we create a "volume" column to see the quantity sold of that item with respect to total sold quantity
majority_items["volume"] = majority_items["quantity"]/majority_items["quantity"].sum()
majority_items.sort_values("volume", ascending=False, inplace=True)

majority_items

,item,quantity,volume
0,TDZ23C,13482,0.000052
1,BSOZV0,12464,0.000048
2,06MGOE,12063,0.000046
3,1JXGFA,11538,0.000044
4,WOS9VG,11475,0.000044
...,...,...,...
49960,RLUDFV,1189,0.000005
49961,7CKFW8,1155,0.000004
49962,CLE0LJ,1121,0.000004
49963,S8142B,802,0.000003


<span style= "color: orange">Now, we are going to compare the first 5k, 10k 15k and 20k rows respectively.</span>

In [14]:
majorities = [majority_items.head(5000), majority_items.head(10000), majority_items.head(15000), majority_items.head(20000)]

## Sum over "volume" column
majority_results = []
for x in majorities:
    majority_results.append(x["volume"].sum().round(3))

majority_results

[0.147, 0.274, 0.39, 0.499]

### <span style= "color: orange">Each part of the data represents the whole data with corresponding ratio:</span>
| Quantity of distinct<br/>items | Represented part<br/>of the full data |
| ------------------------------ | ------------------------------------- |
| 5,000 items                    | 14.7 %                                |
| 10,000 items                   | 27.4 %                                |
| 15,000 items                   | 39.0 %                                |
| 20,000 items                   | 49.9 %                                |
### <span style= "color: orange">We will work with this 10,000 items because in real life cases it will/can represent 90 - 95 % of the full data (this is not the case here because our data is randomly generated but for the sake of the practice, I will use reduced dataset).</span>
### <span style= "color: orange">The reason for this application is that data with 1M row can be too much to deal with because of our available memory space, 16GB or 32GB RAM may not be enough.</span>

In [15]:
majority_active_five_years_table = active_five_years_table[active_five_years_table["item"].isin(majority_items.head(10000)["item"].to_list())].sort_values("date").reset_index(drop=True)
majority_active_five_years_table

,transaction_id,quantity,date,user,item
0,4383,261,2017-06-07,3DX,2HH9BB
1,1324,352,2017-06-07,7IN,VC4ZHH
2,1902,233,2017-06-07,WD6,AD8X2H
3,4498,333,2017-06-07,N7H,9DZSWS
4,4498,273,2017-06-07,0TV,QJJ0DO
...,...,...,...,...,...
264332,1559,142,2021-12-31,S1J,QSHRA9
264333,3064,423,2021-12-31,98M,WUGQSW
264334,3686,153,2021-12-31,5JB,11YGGW
264335,3397,476,2021-12-31,WXR,M2D1J5


<span style= "color: orange">"active_five_years_table" has 1,037,370 (~1M) rows and "majority_active_five_years_table" has 264,337 (~260kM) rows.</br></br>Next, we will save this as CSV for easy import and for our later operations.</span>

In [16]:
majority_active_five_years_table.to_csv("majority_active_five_years_table.csv", index=False)